In [1]:
import numpy  as np
import pandas as pd

In [2]:
df_trn = pd.read_csv('../data/training_le.csv')
df_tst = pd.read_csv('../data/test_le.csv')

In [3]:
print(df_trn.isnull().sum().max())
print(df_tst.isnull().sum().max())

0
0


In [4]:
list(df_trn.columns)

['BatchId',
 'AccountId',
 'SubscriptionId',
 'CustomerId',
 'ProviderId',
 'ProductId',
 'ProductCategory',
 'ChannelId',
 'Amount',
 'Value',
 'TransactionStartTime',
 'PricingStrategy',
 'FraudResult']

In [5]:
# probability encoding - pe
columns4pe = [
    'BatchId',
    'AccountId',
    'SubscriptionId',
    'CustomerId',
    'ProviderId',
    'ProductId',
    'ProductCategory',
    'ChannelId',
    'PricingStrategy'
]

In [6]:
# проверяем как пересекаются поля на train и test
for clm in columns4pe:
    print('=== ', clm, '===')

    print('Train:', min(df_trn[clm]), max(df_trn[clm]), df_trn[clm].nunique())
    print('Test: ', min(df_tst[clm]), max(df_tst[clm]), df_tst[clm].nunique())

    print('Union       :', len(set(df_trn[clm]) | set(df_tst[clm])))
    print('Intersection:', len(set(df_trn[clm]) & set(df_tst[clm])))

===  BatchId ===
Train: 1 139492 94809
Test:  0 139491 44684
Union       : 139493
Intersection: 0
===  AccountId ===
Train: 0 4840 3633
Test:  9 4840 2471
Union       : 4841
Intersection: 1263
===  SubscriptionId ===
Train: 1 4835 3627
Test:  0 4835 2470
Union       : 4836
Intersection: 1261
===  CustomerId ===
Train: 0 7478 3742
Test:  15 7448 5018
Union       : 7479
Intersection: 1281
===  ProviderId ===
Train: 0 5 6
Test:  0 5 6
Union       : 6
Intersection: 6
===  ProductId ===
Train: 0 26 23
Test:  0 26 26
Union       : 27
Intersection: 22
===  ProductCategory ===
Train: 0 9 9
Test:  0 9 9
Union       : 10
Intersection: 8
===  ChannelId ===
Train: 0 4 4
Test:  0 4 5
Union       : 5
Intersection: 4
===  PricingStrategy ===
Train: 0 3 4
Test:  0 3 4
Union       : 4
Intersection: 4


In [7]:
# удаляем BatchID как поле с уникальными значениями
columns4pe.remove('BatchId')

In [8]:
df_trn.head(10)

,BatchId,AccountId,SubscriptionId,CustomerId,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
0,36122,3956,886,4405,5,9,0,2,1000.0,1000,2018-11-15 02:18:49,2,0
1,15641,4840,3828,4405,3,5,2,1,-20.0,20,2018-11-15 02:19:08,2,0
2,53940,4228,221,4682,5,0,0,2,500.0,500,2018-11-15 02:44:21,2,0
3,102362,647,2184,987,0,20,9,2,20000.0,21800,2018-11-15 03:32:55,2,0
4,38779,4840,3828,987,3,5,2,1,-644.0,644,2018-11-15 03:34:21,2,0
5,25953,1077,4237,1431,5,2,0,2,2000.0,2000,2018-11-15 03:35:10,2,0
6,118459,2441,1979,2857,4,2,0,2,10000.0,10000,2018-11-15 03:44:31,3,0
7,38560,4840,3828,2857,3,5,2,1,-500.0,500,2018-11-15 03:45:13,2,0
8,93773,271,4730,597,5,9,0,2,500.0,500,2018-11-15 04:14:59,2,0
9,82408,709,919,1052,0,14,2,2,600.0,600,2018-11-15 04:31:48,2,0


### Попробовать изначально в поля `AccountId`, `SubscriptionId`, `CustomerId` на `test`е занести `-1`
то есть как-то указать, что об этом варианте нам ничего не известно

In [9]:
# for field_name in columns4pe:
#     df_tst[field_name] = -1

# df_tst.head(10)

In [10]:
for clm in columns4pe:
    clm_len = len(set(df_trn[clm]) | set(df_tst[clm]))
    dic = {}
    for i in range(clm_len):
        num_frauds    = df_trn[ (df_trn['FraudResult']==1) & (df_trn[clm]==i) ].shape[0]
        num_no_frauds = df_trn[ (df_trn['FraudResult']==0) & (df_trn[clm]==i) ].shape[0]
        if num_frauds + num_no_frauds != 0:
            dic[i] =  num_frauds / (num_frauds + num_no_frauds)
        else:
            dic[i] = 0.0

    print('=== %s ===' % clm)
    if clm_len < 100:
        for key in dic:
            print(key, ':', dic[key])
        
    df_trn[clm] = df_trn[clm].map(dic)
    df_tst[clm] = df_tst[clm].map(dic)

=== AccountId ===
=== SubscriptionId ===
=== CustomerId ===
=== ProviderId ===
0 : 0.010101010101010102
1 : 0.0
2 : 0.020752269779507133
3 : 0.00013092775406530675
4 : 0.004401045248246458
5 : 8.775522143567542e-05
=== ProductId ===
0 : 0.0
1 : 0.0
2 : 0.0004929346040092015
3 : 0.0
4 : 0.08
5 : 3.064194882794546e-05
6 : 0.0
7 : 0.0
8 : 0.17647058823529413
9 : 0.000390015600624025
10 : 0.0
11 : 0.0
12 : 0.02027027027027027
13 : 0.0
14 : 0.013122701437646272
15 : 0.0
16 : 0.0
17 : 0.0
18 : 0.0
19 : 0.0
20 : 0.0026455026455026454
21 : 0.017857142857142856
22 : 0.0
23 : 0.0
24 : 0.0
25 : 0.0
26 : 0.0
=== ProductCategory ===
0 : 0.0003997601439136518
1 : 0.0
2 : 0.003545864992842198
3 : 0.0
4 : 0.0
5 : 0.0
6 : 0.0
7 : 0.08
8 : 0.0
9 : 0.00625
=== ChannelId ===
0 : 0.007434944237918215
1 : 0.00013462211572117067
2 : 0.0032317555106700624
3 : 0.0
4 : 0.0
=== PricingStrategy ===
0 : 0.09350649350649351
1 : 0.003749330476700589
2 : 0.0017408075343151989
3 : 0.0008110898097625719


In [11]:
df_trn.head(10)

,BatchId,AccountId,SubscriptionId,CustomerId,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
0,36122,0.000000,0.000000,0.0,0.000088,0.000390,0.000400,0.003232,1000.0,1000,2018-11-15 02:18:49,0.001741,0
1,15641,0.000032,0.000031,0.0,0.000131,0.000031,0.003546,0.000135,-20.0,20,2018-11-15 02:19:08,0.001741,0
2,53940,0.000000,0.000000,0.0,0.000088,0.000000,0.000400,0.003232,500.0,500,2018-11-15 02:44:21,0.001741,0
3,102362,0.000000,0.000000,0.0,0.010101,0.002646,0.006250,0.003232,20000.0,21800,2018-11-15 03:32:55,0.001741,0
4,38779,0.000032,0.000031,0.0,0.000131,0.000031,0.003546,0.000135,-644.0,644,2018-11-15 03:34:21,0.001741,0
5,25953,0.000000,0.000000,0.0,0.000088,0.000493,0.000400,0.003232,2000.0,2000,2018-11-15 03:35:10,0.001741,0
6,118459,0.000000,0.000000,0.0,0.004401,0.000493,0.000400,0.003232,10000.0,10000,2018-11-15 03:44:31,0.000811,0
7,38560,0.000032,0.000031,0.0,0.000131,0.000031,0.003546,0.000135,-500.0,500,2018-11-15 03:45:13,0.001741,0
8,93773,0.000000,0.000000,0.0,0.000088,0.000390,0.000400,0.003232,500.0,500,2018-11-15 04:14:59,0.001741,0
9,82408,0.000000,0.000000,0.0,0.010101,0.013123,0.003546,0.003232,600.0,600,2018-11-15 04:31:48,0.001741,0


In [12]:
df_tst.head(10)

,BatchId,AccountId,SubscriptionId,CustomerId,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy
0,35027,0.000000,0.000000,0.0,0.004401,0.000493,0.000400,0.003232,1000.0,1000,2019-02-13 10:01:40,0.000811
1,45138,0.000000,0.000000,0.0,0.004401,0.013123,0.003546,0.003232,2000.0,2000,2019-02-13 10:02:12,0.001741
2,74886,0.000032,0.000031,0.0,0.000131,0.000031,0.003546,0.000135,-50.0,50,2019-02-13 10:02:30,0.001741
3,11024,0.000000,0.000000,0.0,0.004401,0.000390,0.000400,0.003232,3000.0,3000,2019-02-13 10:02:38,0.000811
4,29803,0.000032,0.000031,0.0,0.000131,0.000031,0.003546,0.000135,-60.0,60,2019-02-13 10:02:58,0.001741
5,67574,0.000000,0.000000,0.0,0.000088,0.000493,0.000400,0.003232,1000.0,1000,2019-02-13 10:03:12,0.001741
6,52374,0.000032,0.000031,0.0,0.000131,0.000031,0.003546,0.000135,-50.0,50,2019-02-13 10:03:48,0.001741
7,112551,0.000000,0.000000,0.0,0.020752,0.000493,0.000400,0.003232,1000.0,1000,2019-02-13 10:09:46,0.003749
8,60866,0.000032,0.000031,0.0,0.000131,0.000031,0.003546,0.000135,-50.0,50,2019-02-13 10:11:24,0.001741
9,14720,0.000000,0.000000,0.0,0.020752,0.000000,0.003546,0.003232,1161.0,1161,2019-02-13 10:12:08,0.093506


In [13]:
print(df_trn.isnull().sum())
print()
print(df_tst.isnull().sum())

BatchId                 0
AccountId               0
SubscriptionId          0
CustomerId              0
ProviderId              0
ProductId               0
ProductCategory         0
ChannelId               0
Amount                  0
Value                   0
TransactionStartTime    0
PricingStrategy         0
FraudResult             0
dtype: int64

BatchId                 0
AccountId               0
SubscriptionId          0
CustomerId              0
ProviderId              0
ProductId               0
ProductCategory         0
ChannelId               0
Amount                  0
Value                   0
TransactionStartTime    0
PricingStrategy         0
dtype: int64


In [14]:
# Detect missing values (NaN or None)
print(df_trn.isna().count())
print()
print(df_tst.isna().count())

BatchId                 95662
AccountId               95662
SubscriptionId          95662
CustomerId              95662
ProviderId              95662
ProductId               95662
ProductCategory         95662
ChannelId               95662
Amount                  95662
Value                   95662
TransactionStartTime    95662
PricingStrategy         95662
FraudResult             95662
dtype: int64

BatchId                 45019
AccountId               45019
SubscriptionId          45019
CustomerId              45019
ProviderId              45019
ProductId               45019
ProductCategory         45019
ChannelId               45019
Amount                  45019
Value                   45019
TransactionStartTime    45019
PricingStrategy         45019
dtype: int64


In [15]:
# Indicator whether DataFrame is empty
print(df_trn.empty, df_tst.empty)

False False


In [27]:
# for i in range(30):
#     print('%.5f' % df_trn['CustomerId'][i])

0.00000
0.00000
0.00000
0.00000
0.00000
0.00000
0.00000
0.00000
0.00000
0.00000
0.00000
0.00000
0.00000
0.00000
0.00000
0.00000
0.00000
0.00000
0.00000
0.00000
0.00000
0.00000
0.00000
0.00847
0.00000
0.00000
0.00847
0.00000
0.00000
0.00000


In [28]:
df_trn.to_csv('../data/train_pe.csv', encoding='utf-8', index=False)
df_tst.to_csv('../data/test_pe.csv',  encoding='utf-8', index=False)

In [ ]:
df_trn[df_trn['FraudResult']==1]

In [ ]:
df_trn[df_trn['FraudResult']==1].groupby('ProviderId')['ProviderId'].count()

In [ ]:
df_trn.groupby(['ProductId', 'FraudResult'])['FraudResult'].count()

In [ ]:
df_trn[ (df_trn['ProductId']==0) & (df_trn['FraudResult']==1) ].shape[0]

In [ ]:
df_trn.groupby(['ProviderId', 'FraudResult'])['ProviderId'].count()

In [ ]:
df_trn['ProviderId'].value_counts()

In [ ]:
pvt = df.pivot_table(index=['FraudResults'], columns=['ProductId'], values='TransactionID', aggfunc='count')

In [ ]:
df.groupby(['FraudResults'])